<a href="https://colab.research.google.com/github/dhwanipatel14/Comp-4730-Project-1/blob/master/VGG_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
!pip install keras
import keras,os
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten
from keras.preprocessing.image import ImageDataGenerator
import numpy as np

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [16]:
#importing all the essential libraries 
import tensorflow as tf
import numpy as np
import numpy as np 
import pandas as pd 
import os, time
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import keras
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Dense, Dropout, Flatten
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import VGG16;
from keras.applications.vgg16 import preprocess_input
import os

In [17]:
#loading mnist dataset from keras
mnist = tf.keras.datasets.mnist

In [18]:
#trainX is an array, etc, arraying from 0-255
(trainX, trainY), (testX, testY) = mnist.load_data()
print(trainX.shape, testX.shape)

(60000, 28, 28) (10000, 28, 28)


In [19]:
trainX=np.dstack([trainX] * 3)
testX=np.dstack([testX]*3)
trainX.shape,testX.shape

((60000, 28, 84), (10000, 28, 84))

In [20]:
#reshape images as per the tensor format required by tensorflow
trainX = trainX.reshape(-1, 28,28,3)
testX= testX.reshape (-1,28,28,3)
trainX.shape,testX.shape

((60000, 28, 28, 3), (10000, 28, 28, 3))

In [22]:
#resize the images 48*48 as required by VGG16
from tensorflow.keras.utils import img_to_array, array_to_img
trainX = np.asarray([img_to_array(array_to_img(im, scale=False).resize((48,48))) for im in trainX])
testX = np.asarray([img_to_array(array_to_img(im, scale=False).resize((48,48))) for im in testX])

#trainx = preprocess_input(x)
trainX.shape, testX.shape
classes = np.unique(trainY)
num_classes = len(classes)
num_classes

10

In [23]:
# Normalise the data and change data type
trainX = trainX / 255.
testX = testX / 255.
trainX = trainX.astype('float32')
testX = testX.astype('float32')

In [24]:
#converting Labels to one hot encoded format
trainY_one_hot = to_categorical(trainY)
testY_one_hot = to_categorical(testY)

trainX,valid_X,train_label,valid_label = train_test_split(trainX, trainY_one_hot,test_size=0.2,random_state=13)

In [27]:
#finally check the data size whether it is as per tensorflow and VGG16 requirement
trainX.shape,valid_X.shape,train_label.shape,valid_label.shape

#define the parameters for instanitaing VGG16 model. 
IMG_WIDTH = 48
IMG_HEIGHT = 48
IMG_DEPTH = 3
BATCH_SIZE = 16

#preprocessing the input 
trainX = preprocess_input(trainX)
valid_X = preprocess_input(valid_X)
testX  = preprocess_input (testX)

#create base model of VGG16
conv_base = VGG16(weights='imagenet',include_top=False, input_shape=(IMG_HEIGHT, IMG_WIDTH, IMG_DEPTH))
conv_base.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 48, 48, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 48, 48, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 48, 48, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 24, 24, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 24, 24, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 24, 24, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 12, 12, 128)       0     

In [26]:
#extracting features
train_features = conv_base.predict(np.array(trainX), batch_size=BATCH_SIZE, verbose=1)
test_features = conv_base.predict(np.array(testX), batch_size=BATCH_SIZE, verbose=1)
val_features = conv_base.predict(np.array(valid_X), batch_size=BATCH_SIZE, verbose=1)

750/750 [==============================] - 302s 403ms/step


In [28]:
#saving the features so that they can be used for future
np.savez("train_features", train_features, train_label)
np.savez("test_features", test_features, testY)
np.savez("val_features", val_features, valid_label)

In [29]:
train_features_flat = np.reshape(train_features, (48000, 1*1*512))
test_features_flat = np.reshape(test_features, (10000, 1*1*512))
val_features_flat = np.reshape(val_features, (12000, 1*1*512))

In [31]:
from keras import models
from keras.models import Model
from keras import layers
from keras import optimizers
from keras import callbacks
from keras.layers import ELU, PReLU, LeakyReLU

In [32]:
NB_TRAIN_SAMPLES = train_features_flat.shape[0]
NB_VALIDATION_SAMPLES = val_features_flat.shape[0]
NB_EPOCHS = 100

model = models.Sequential()
model.add(layers.Dense(512, activation='relu', input_dim=(1*1*512)))
model.add(layers.LeakyReLU(alpha=0.1))
model.add(layers.Dense(num_classes, activation='softmax'))

In [33]:
model.compile(loss='categorical_crossentropy',optimizer=optimizers.Adam(),metrics=['acc'])

In [34]:
reduce_learning = callbacks.ReduceLROnPlateau(monitor='val_loss',factor=0.2,patience=2,verbose=1,mode='auto',epsilon=0.0001,cooldown=2,min_lr=0)
eary_stopping = callbacks.EarlyStopping(monitor='val_loss',min_delta=0,patience=7,verbose=1,mode='auto')
callbacks = [reduce_learning, eary_stopping]

In [35]:
#train the model
history = model.fit(train_features_flat,train_label,epochs=NB_EPOCHS,validation_data=(val_features_flat, valid_label),callbacks=callbacks)

Epoch 1/100
1500/1500 [==============================] - 10s 6ms/step - loss: 1.4070 - acc: 0.4982 - val_loss: 1.0905 - val_acc: 0.5992 - lr: 0.0010
Epoch 2/100
1500/1500 [==============================] - 8s 6ms/step - loss: 1.0011 - acc: 0.6505 - val_loss: 0.9357 - val_acc: 0.6780 - lr: 0.0010
Epoch 3/100
1500/1500 [==============================] - 8s 5ms/step - loss: 0.8788 - acc: 0.7013 - val_loss: 0.8019 - val_acc: 0.7268 - lr: 0.0010
Epoch 4/100
1500/1500 [==============================] - 8s 5ms/step - loss: 0.8149 - acc: 0.7218 - val_loss: 0.7273 - val_acc: 0.7654 - lr: 0.0010
Epoch 5/100
1500/1500 [==============================] - 8s 6ms/step - loss: 0.7700 - acc: 0.7385 - val_loss: 0.7162 - val_acc: 0.7591 - lr: 0.0010
Epoch 6/100
1500/1500 [==============================] - 8s 6ms/step - loss: 0.7364 - acc: 0.7497 - val_loss: 0.7192 - val_acc: 0.7543 - lr: 0.0010
Epoch 7/100
1500/1500 [==============================] - 9s 6ms/step - loss: 0.7075 - acc: 0.7599 - val_loss: 0